In [4]:
import torch
import torch.nn as nn
from utils.utils import *
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
from utils.utils import *
plt.ion()   # interactive mode
device = 'cpu'

### Computing the inception score with the KL-divergence formula

In [13]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3

import numpy as np
from scipy.stats import entropy

def inception_score(gen,N, wavegan,cuda=True, batch_size=32):
    
    N=(N//32)*32
    
    # loading the trained inceptionv3 model
    model_ft = models.inception_v3(pretrained=True)
    model_ft.aux_logits=False
    num_ftrs = model_ft.fc.in_features

    model_ft.fc = nn.Linear(num_ftrs, 10)

    model_ft = model_ft.to(device)
    # loading the trained inceptionv3 model weights
    model_ft.load_state_dict(torch.load('./data/INCEPTIONV3.pt',map_location=device))
    model_ft.eval()

    # same transf as in the inceptionv3 training
    data_transforms = torchvision.transforms.Compose([
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.Resize(299),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # get inceptionv3 output from the generated clips X
    def get_pred(x):
        
        #if wavegan get the spectrogram images
        if wavegan:
            imgs=[]
            for sample in x:
                imgs.append(data_transforms(transform_spectrogram(sample).detach().repeat(3, 1, 1))) #add 3 channels

            x=torch.from_numpy(np.stack(imgs)) 
        else:
            imgs=[]
            for img in x:
                imgs.append(data_transforms(img.repeat(3, 1, 1))) #add 3 channels

            x=torch.from_numpy(np.stack(imgs)) 
            
        
        x = model_ft(x)
        return F.softmax(x).data.cpu().numpy() # inception output

    # Get predictions
    preds = np.zeros((N, 10))

    
    # get the inceptionv3 output for eqch batch
    for i in range(N//batch_size):
        print('step',i+1,'/',N//batch_size)
        noise = torch.randn(batch_size, 100).to(device)
        fake_audio = gen(noise)
        preds[i*batch_size:i*batch_size + batch_size] = get_pred(fake_audio)
        
        
    #kl-div, compute score
    split_scores = []

    
    part = preds[0: N, :]
    py = np.mean(part, axis=0)
    scores = []
    for i in range(part.shape[0]):
        pyx = part[i, :]
        scores.append(entropy(pyx, py))
    split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores)

### Inception score for WaveGAN

In [18]:
from utils.wavegan import *

#loading the WaveGan generator
wave_gen = WaveGenerator(d=64, c=1 ,inplace=True).to(device)
wave_gen.eval()
wave_gen.load_state_dict(torch.load('./data/gen_180_66970.pt',map_location=device))

#inception score
inception_score(wave_gen,1000,wavegan=True)

step 1 / 31


C:\Users\nguye\anaconda3\envs\envDL\lib\site-packages\ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


step 2 / 31
step 3 / 31
step 4 / 31
step 5 / 31
step 6 / 31
step 7 / 31
step 8 / 31
step 9 / 31
step 10 / 31
step 11 / 31
step 12 / 31
step 13 / 31
step 14 / 31
step 15 / 31
step 16 / 31
step 17 / 31
step 18 / 31
step 19 / 31
step 20 / 31
step 21 / 31
step 22 / 31
step 23 / 31
step 24 / 31
step 25 / 31
step 26 / 31
step 27 / 31
step 28 / 31
step 29 / 31
step 30 / 31
step 31 / 31


3.768076757610188

### Inception score for SpecGAN

In [17]:
from utils.specgan import *

#loading the SpecGAN generator
specgan = SpecGenerator(d=64, c=1 ,inplace=True).to(device)
specgan.eval()
specgan.load_state_dict(torch.load('./data/gen_96_71780.pt',map_location=device))

#inception score
inception_score(specgan,1000,wavegan=False)

step 1 / 31


C:\Users\nguye\anaconda3\envs\envDL\lib\site-packages\ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


step 2 / 31
step 3 / 31
step 4 / 31
step 5 / 31
step 6 / 31
step 7 / 31
step 8 / 31
step 9 / 31
step 10 / 31
step 11 / 31
step 12 / 31
step 13 / 31
step 14 / 31
step 15 / 31
step 16 / 31
step 17 / 31
step 18 / 31
step 19 / 31
step 20 / 31
step 21 / 31
step 22 / 31
step 23 / 31
step 24 / 31
step 25 / 31
step 26 / 31
step 27 / 31
step 28 / 31
step 29 / 31
step 30 / 31
step 31 / 31


1.483256607675118